In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,135 kB]
Get:13 http://security.ubuntu.com/ubuntu ja

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()


In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()


+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [4]:
# What occupation had the highest salary?
df.orderBy(df["Salary"].desc()).select("occupation", "Salary").limit(1).show()


+-----------------+------+
|       occupation|Salary|
+-----------------+------+
|Chemical Engineer|   120|
+-----------------+------+



In [5]:
# What occupation had the lowest salary?
df.orderBy(df["Salary"]).select("occupation", "Salary").limit(1).show()


+----------------+------+
|      occupation|Salary|
+----------------+------+
|Product Engineer|    40|
+----------------+------+



In [6]:
# What is the average salary of this dataset?
from pyspark.sql.functions import avg
df.select(avg("Salary")).show()


+-----------+
|avg(Salary)|
+-----------+
|     79.475|
+-----------+



In [7]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min
df.select(max("Salary"), min("Salary")).show()


+-----------+-----------+
|max(Salary)|min(Salary)|
+-----------+-----------+
|        120|         40|
+-----------+-----------+



In [8]:
# Show all of the occupations where salaries were above 80k
from pyspark.sql.functions import count
df.filter("Salary > 80").select("occupation").show()


+--------------------+
|          occupation|
+--------------------+
|   Account Executive|
|Sales Representative|
|        Developer II|
|     Design Engineer|
|     Project Manager|
|       Programmer II|
|Sales Representative|
|Physical Therapy ...|
|Payment Adjustmen...|
|Mechanical System...|
|     Media Manager I|
|   Account Executive|
|           Professor|
|Community Outreac...|
| Clinical Specialist|
|Human Resources A...|
|Nuclear Power Eng...|
|      Civil Engineer|
|Human Resources M...|
|Senior Cost Accou...|
+--------------------+
only showing top 20 rows



In [9]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
avg_df = df.groupBy("academic_degree").avg()
avg_df.select("academic_degree", "avg(age)", "avg(height_m)").show()


+---------------+------------------+------------------+
|academic_degree|          avg(age)|     avg(height_m)|
+---------------+------------------+------------------+
|            PhD| 42.87818696883853|1.7537393767705372|
|         Master|42.105095541401276|1.7606050955414014|
|       Bachelor| 43.85585585585586|1.7371771771771771|
+---------------+------------------+------------------+



In [11]:
# What occupation had the highest salary?
df.orderBy(df["Salary"].desc()).select("occupation", "Salary").limit(1).show()

+-----------------+------+
|       occupation|Salary|
+-----------------+------+
|Chemical Engineer|   120|
+-----------------+------+



In [12]:
# What occupation had the lowest salary?
df.orderBy(df["Salary"]).select("occupation", "Salary").limit(1).show()

+----------------+------+
|      occupation|Salary|
+----------------+------+
|Product Engineer|    40|
+----------------+------+



In [13]:
# What is the average salary of this dataset?
from pyspark.sql.functions import avg
df.select(avg("Salary")).show()

+-----------+
|avg(Salary)|
+-----------+
|     79.475|
+-----------+



In [14]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min
df.select(max("Salary"), min("Salary")).show()

+-----------+-----------+
|max(Salary)|min(Salary)|
+-----------+-----------+
|        120|         40|
+-----------+-----------+



In [15]:
# Show all of the occupations where salaries were above 80k
from pyspark.sql.functions import count
df.filter("Salary > 80").select("occupation").show()

+--------------------+
|          occupation|
+--------------------+
|   Account Executive|
|Sales Representative|
|        Developer II|
|     Design Engineer|
|     Project Manager|
|       Programmer II|
|Sales Representative|
|Physical Therapy ...|
|Payment Adjustmen...|
|Mechanical System...|
|     Media Manager I|
|   Account Executive|
|           Professor|
|Community Outreac...|
| Clinical Specialist|
|Human Resources A...|
|Nuclear Power Eng...|
|      Civil Engineer|
|Human Resources M...|
|Senior Cost Accou...|
+--------------------+
only showing top 20 rows



In [17]:
# Displaying the number of rows that satisfy the condition.
print("Total count:", df.count())

Total count: 1000


In [18]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
avg_df = df.groupBy("academic_degree").avg()
avg_df.select("academic_degree", "avg(age)", "avg(height_m)").show()

+---------------+------------------+------------------+
|academic_degree|          avg(age)|     avg(height_m)|
+---------------+------------------+------------------+
|            PhD| 42.87818696883853|1.7537393767705372|
|         Master|42.105095541401276|1.7606050955414014|
|       Bachelor| 43.85585585585586|1.7371771771771771|
+---------------+------------------+------------------+

